In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import time

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'
if not os.path.exists(volume_dir):
    os.makedirs(volume_dir)

In [4]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'

In [5]:
scoremaps_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremaps_lossless_svm_Sat16ClassFinetuned_v3/'

In [6]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

colors = np.random.randint(0, 255, (len(labels_index), 3))

In [7]:
downsample_factor = 16

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

In [8]:
##### reconstruct score volume ######

In [8]:
from joblib import Parallel, delayed

In [9]:
def load_scoremap(sec, l):
    
    scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.hdf' \
            % {'stack': stack, 'slice': sec, 'label': l}
        
    if not os.path.exists(scoremap_bp_filepath):
        sys.stderr.write('No scoremap for section %d\n' % sec)
        return None

    scoremap_roi = load_hdf(scoremap_bp_filepath)

    interpolation_xmin, interpolation_xmax, \
    interpolation_ymin, interpolation_ymax = np.loadtxt(scoremaps_rootdir + \
                        '/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                    {'stack': stack, 'sec': sec, 'label': l}).astype(np.int)

    dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)

    dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                            interpolation_xmin:interpolation_xmax+1] = scoremap_roi
    scoremap = dense_scoremap_lossless[::downsample_factor, ::downsample_factor].copy()
    
    del scoremap_roi, dense_scoremap_lossless

    return scoremap

In [ ]:
# t = time.time()

# W = Parallel(n_jobs=16)(delayed(load_scoremap)(sec, l) for sec in range(section_roi_begin, section_roi_end+1))

# volume = np.zeros((dm.image_height/downsample_factor, 
#                            dm.image_width/downsample_factor, 
#                            int(section_number_lookup[stack]*z_xy_ratio_downsampled)), np.float32)

# for sec in range(section_roi_begin, section_roi_end+1):
#     volume[:, :, int(sec*z_xy_ratio_downsampled):int((sec+1)*z_xy_ratio_downsampled)] = W[sec-section_roi_begin][..., None]

# sys.stderr.write('build volume: %.2f seconds\n' % (time.time() - t))

In [ ]:
# stack = 'MD594'
# stack = 'MD593'

for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
# for stack in ['MD594']:
    
    print stack

    section_roi_begin, section_roi_end = detect_bbox_range_lookup[stack]
    print section_roi_begin, section_roi_end

    dm = DataManager(stack=stack)

    roi_zmin = int(section_roi_begin*z_xy_ratio_downsampled)
    roi_zmax = int((section_roi_end+1)*z_xy_ratio_downsampled)-1
    print roi_zmin, roi_zmax

    volume_xmin, volume_ymin, volume_zmin = (np.inf, np.inf, np.inf)
    volume_xmax, volume_ymax, volume_zmax = (0,0,0)

    t = time.time()
    
    for l in labels[1:]:

        print l

        volume = np.zeros((dm.image_height/downsample_factor, 
                           dm.image_width/downsample_factor, 
                           int(section_number_lookup[stack]*z_xy_ratio_downsampled)), np.float32)
        print volume.shape

        
        t1 = time.time()
        
#         for sec in range(section_roi_begin, section_roi_end+1):
            
#             scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.hdf' \
#             % {'stack': stack, 'slice': sec, 'label': l}
        
#             if not os.path.exists(scoremap_bp_filepath):
#                 sys.stderr.write('No scoremap for section %d\n' % sec)
#                 continue

# #             print sec

#             scoremap_roi = load_hdf(scoremap_bp_filepath)


#             dataset = stack + '_' + '%04d'%sec + '_roi1'

#             interpolation_xmin, interpolation_xmax, \
#             interpolation_ymin, interpolation_ymax = np.loadtxt(scoremaps_rootdir + \
#                                 '/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s_interpBox.txt' % \
#                                             {'stack': stack, 'sec': sec, 'label': l}).astype(np.int)

#             dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)

#             dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
#                                     interpolation_xmin:interpolation_xmax+1] = scoremap_roi
#             scoremap = dense_scoremap_lossless[::downsample_factor, ::downsample_factor]
#             volume[:, :, int(sec*z_xy_ratio_downsampled):int((sec+1)*z_xy_ratio_downsampled)] = scoremap[..., np.newaxis].copy()

#             del scoremap, scoremap_roi, dense_scoremap_lossless

        W = Parallel(n_jobs=16)(delayed(load_scoremap)(sec, l) for sec in range(section_roi_begin, section_roi_end+1))
        
        for sec in range(section_roi_begin, section_roi_end+1):
            if W[sec-section_roi_begin] is None:
                continue
            volume[:, :, int(sec*z_xy_ratio_downsampled):int((sec+1)*z_xy_ratio_downsampled)] = W[sec-section_roi_begin][..., None]

        sys.stderr.write('build volume: %.2f seconds\n' % (time.time() - t1)) # ~1.5s / section
    
        ys, xs, zs = np.where(volume > 0)

        xmin = xs.min()
        xmax = xs.max()
        ymin = ys.min()
        ymax = ys.max()
        zmin = zs.min()
        zmax = zs.max()

        print xmin, xmax
        print ymin, ymax
        print zmin, zmax

        if xmin < volume_xmin:
            volume_xmin = xmin
        if ymin < volume_ymin:
            volume_ymin = ymin
        if zmin < volume_zmin:
            volume_zmin = zmin
        if xmax > volume_xmax:
            volume_xmax = xmax
        if ymax > volume_ymax:
            volume_ymax = ymax
        if zmax > volume_zmax:
            volume_zmax = zmax

        bp.pack_ndarray_file(volume[..., roi_zmin:roi_zmax+1],
                     os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_%(label)s.bp' % {'stack': stack, 'label': l}))    

        del volume, ys, xs, zs, W

    sys.stderr.write('build volume all: %.2f seconds\n' % (time.time() - t))
    
    print volume_xmin, volume_ymin, volume_zmin
    print volume_xmax, volume_ymax, volume_zmax

    with open(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_limits.txt' % {'stack': stack}), 'w') as f:
        f.write((' '.join(['%d']*6)+'\n') % (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax))

MD585
132 292
358 795
5N
(816, 890, 1195)
35 850
291 735
358 795
7n
(816, 890, 1195)


build volume: 24.05 seconds


35 850
291 735
358 795
7N
(816, 890, 1195)


build volume: 19.60 seconds
No scoremap for section 179
No scoremap for section 181
No scoremap for section 196
No scoremap for section 198
No scoremap for section 202
No scoremap for section 204
No scoremap for section 222
No scoremap for section 224
No scoremap for section 226
No scoremap for section 232
No scoremap for section 263
No scoremap for section 182
No scoremap for section 228
No scoremap for section 230
No scoremap for section 183
No scoremap for section 184
No scoremap for section 185
No scoremap for section 186
No scoremap for section 187
No scoremap for section 188
No scoremap for section 189
No scoremap for section 190
No scoremap for section 191
No scoremap for section 192
No scoremap for section 193
No scoremap for section 194
